# Installation des dépendances

Pour pouvoir exécuter ce Notebook, l'environnement de développement doit être bien configuré. Puisque dans ce POC nous utilisons YoloV5, il faut installer de nombreuses dépendances dont PyTorch.

<div class="alert alert-info">
    Afin d'exécuter ce Notebook sur AWS SageMaker, il faut utiliser le kernel <code>conda_python3</code>.
</div>

<div class="alert alert-info">
    Il faut compter environ 5 minutes pour l'installation des dépendances.
</div>

In [1]:
!git clone https://github.com/rkuo2000/yolov5

Cloning into 'yolov5'...


On installe ensuite les dépendances pour YoloV5.

In [2]:
!pip install -q -r yolov5/requirements.txt

Enfin, nous installons les dépendances liées aux frameworks Deep Learning (ici PyTorch).

<div class="alert alert-warning">
    Attention, il faut utiliser la bonne version de PyTorch : sur SageMaker, c'est la version <code>1.11.0+cu113</code> mais sur un autre environment, cela peut être différent (notamment sur ton PC ou sur GOogle Colab) !
</div>

In [ ]:
!pip install -q \
    torch==1.11.0+cu113 \
    torchvision==0.12.0+cu113 \
    torchaudio==0.11.0 \
    gcc7 \
    opendatasets \
    pycocotools \
    split-folders \
    --extra-index-url https://download.pytorch.org/whl/cu113

In [ ]:
# Ceci est une petite correction nécessaire pour YoloV5
!pip install -q --upgrade scipy

# Analyse exploratoire des données

Dans cette partie, nous allons étudier le jeu de données. Commençons par récupérer les données en les téléchargeant directement depuis Kaggle.

<div class="alert alert-info">
    Il est nécessaire de créer un compte sur <a href="https://kaggle.com" target="_blank">Kaggle</a> pour télécharger les données.
</div>

In [ ]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/kneroma/tacotrashdataset")

Dans un premier temps, nous allons **comptabiliser le nombre d'images, de catégories de déchets et d'annotations** présentes dans le jeu de données.

### ➡️ À toi de jouer

En parcourant le jeu de données, tu dois calculer :

- le nombre de catégories dans la variable `nr_cats` ;
- le nombre d'annotations dans la variable `nr_annotations` ;
- le nombre d'images dans la variable `nr_images`.

In [ ]:
import os
import shutil
import tqdm
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Style graphique de Seaborn
sns.set()

# Normalement, les données doivent se trouver dans le même dossier que le Notebook
dataset_path = "./tacotrashdataset/data"
anns_file_path = os.path.join(dataset_path, "annotations.json")

# On récupère toutes les annotations
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())

# On calcule le nombre d'images, d'annotations, de catégories de déchets
# TODO
# ...

print("Nombre de catégories :", nr_cats)
print("Nombre d'annotations :", nr_annotations)
print("Nombre d'images :", nr_images)

### ➡️ À toi de jouer

En itérant sur chaque catégorie, tu dois calculer le nombre de super catégories dans la variable `nr_super_cats`.

In [ ]:
# On calcule le nombre de super catégories dans le dataset et on extrait les categories dans une liste
cat_names = []
super_cat_names = []
super_cat_ids = {}
super_cat_last_name = ''
nr_super_cats = 0

for cat_it in categories:
    # TODO
    # ...

print("Nombre de super catégories :", nr_super_cats)

Maintenant, essayons de déterminer le nombre d'annotations pour chaque super catégorie.

### ➡️ À toi de jouer

Tracer sur un graphique (avec Matplotlib ou Seaborn) le nombre d'annotations pour chaque **super catégorie**.

In [ ]:
# On détermine les annotations pour chaque super catégorie
cat_ids_2_supercat_ids = {}
for cat in categories:
    cat_ids_2_supercat_ids[cat['id']] = super_cat_ids[cat['supercategory']]

# On compte les annotations
# TODO
# ...
    
# Initialiser la figure matplotlib
f, ax = plt.subplots(figsize=(5,10))

# Conversion à un DataFrame et tri
d = {"Super catégories": super_cat_names, "Nombre d'annotations": super_cat_histogram}
df = pd.DataFrame(d)
df = df.sort_values("Nombre d'annotations", 0, False)

# Affichage
# TODO
# ...
plt.title("Nombre d'annotations pour chaque catégorie")
plt.show()

### ➡️ À toi de jouer

Tracer sur un graphique (avec Matplotlib ou Seaborn) le nombre d'annotations pour chaque **catégorie**.

In [ ]:
# Même traitement mais pour les catégories
cat_histogram = np.zeros(nr_cats,dtype=int)

# TODO
# ...

f, ax = plt.subplots(figsize=(5, 15))
df = pd.DataFrame({"Catégories": cat_names, "Nombre d'annotations": cat_histogram})
df = df.sort_values("Nombre d'annotations", 0, False)

# TODO
# ...
plt.title("Nombre de super catégories pour chaque catégorie")
plt.show()

Ce que nous montre le graphique, c'est que pour certaines annotations, **il n'y a que très peu d'exemples** : cela peut alors perturber le modèle qui peut avoir du mal à effectuer la classification. Il faudra potentiellement évaluer le modèle que sur les annotations les plus représentées.

Maintenant, il est **important de bien visualiser les annotations sur les images** : cela nous permettra par exemple de savoir quelle métrique nous allons utiliser pour vérifier la qualité du modèle que nous allons construire.

Pour cela, nous allons prendre des images aléatoirement selon le type de déchets qu'elles contiennent et afficher leurs bounding boxes. Pour cela, nous allons prendre comme exemple la catégorie `Clear plastic bottle`.

In [ ]:
import colorsys
import random
import pylab

from PIL import Image, ExifTags
from pycocotools.coco import COCO
from matplotlib.patches import Polygon, Rectangle
from matplotlib.collections import PatchCollection

pylab.rcParams['figure.figsize'] = (14,14)

# Le nombre d'images à afficher
nr_img_2_display = 1

# La catégorie à afficher
category_name = 'Clear plastic bottle'

# On cherche l'orientation d'une image (car elle peut être à l'envers dans certains cas)
for orientation in ExifTags.TAGS.keys():
    if ExifTags.TAGS[orientation] == 'Orientation':
        break
        
# On charge les données (annotations) en tant qu'objet COCO 
coco = COCO(anns_file_path)

# On récupère l'identifiant la catégorie de déchets choisie
imgIds = []
catIds = coco.getCatIds(catNms=[category_name])

if catIds:
    # On cherche les images contenant la catégorie de déchets choisie
    imgIds = coco.getImgIds(catIds=catIds)
else:
    # On cherche les images contenant la super catégorie de déchets choisie si on ne retrouve rien avec les catégories
    catIds = coco.getCatIds(supNms=[category_name])
    for catId in catIds:
        imgIds += (coco.getImgIds(catIds=catId))
    imgIds = list(set(imgIds))

# On affiche combien d'images ont été trouvées
nr_images_found = len(imgIds) 
print("Nombre d'images trouvées :", nr_images_found)

### ➡️ À toi de jouer

Pour chaque image tirée aléatoirement (stockées dans la variable `imgs`), afficher l'image ainsi que la segmentation et la bounding box de chaque annotation de type `Clear plastic bottle` sur l'image.

In [ ]:
# On sélectionne N images aléatoires
random.shuffle(imgIds)
imgs = coco.loadImgs(imgIds[0:min(nr_img_2_display,nr_images_found)])

for img in imgs:
    image_path = dataset_path + '/' + img['file_name']
    # On charge l'image
    I = Image.open(image_path)
    if I._getexif():
        exif = dict(I._getexif().items())
        # Rotation de l'image si nécessaire
        if orientation in exif:
            if exif[orientation] == 3:
                I = I.rotate(180,expand=True)
            if exif[orientation] == 6:
                I = I.rotate(270,expand=True)
            if exif[orientation] == 8:
                I = I.rotate(90,expand=True)
    
    # Affichage de l'image
    fig,ax = plt.subplots(1)
    plt.axis('off')
    plt.imshow(I)

    # Chargement des annotations 
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns_sel = coco.loadAnns(annIds)
    
    # Affichage des segmentations et des bounding boxes à partir des annotations
    for ann in anns_sel:
        # TODO
        # ...

    plt.show()

On voit à la fois la segmentation (en trait plein) et la bounding box (en pointillées).